In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Microphone Locations
mic_height = 50.3125
feet = 12
y_tile = 23.5
x_tile = 11 +7/8

camera_origin_location = np.array([-6*x_tile-5.75, -y_tile, 45+13/16])*25.4

mic_1 = np.array([-11*x_tile - 1, -5*y_tile - 6-3/8, mic_height]) * 25.4 
mic_4= np.array([-11*x_tile + 1.25 + 1/16, 2.5*y_tile+3.75, mic_height]) * 25.4
mic_6 = np.array([5+3/8, 2+1/8, mic_height]) * 25.4
mic_9 = np.array([-3, -6*y_tile - 0.5, mic_height]) * 25.4

mic_xyzs = np.stack((mic_1,mic_4, mic_6, mic_9),axis=0)

In [ ]:
SPEAKER_BOTTOM_RIGHT_Y = (1200.15 + 1196.975 + 1206.5)/ 3
SPEAKER_BOTTOM_RIGHT_X = (88.9 + 107.95 + 101.6) / 3
SPEAKER_BOTTOM_LEFT_Y = (1327.15 + 1311.55712764 + 1317.625) / 3
SPEAKER_BOTTOM_LEFT_X = - 76.98583188

speaker_xyz_bottom_right = np.array([SPEAKER_BOTTOM_RIGHT_X, SPEAKER_BOTTOM_RIGHT_Y, 44.5*25.4])
speaker_xyz_bottom_left = np.array([SPEAKER_BOTTOM_LEFT_X, SPEAKER_BOTTOM_LEFT_Y, 44.5*25.4])
speaker_xyz_top_right = np.array([SPEAKER_BOTTOM_RIGHT_X, SPEAKER_BOTTOM_RIGHT_Y, (44.5+17)*25.4])
speaker_xyz_top_left = np.array([SPEAKER_BOTTOM_LEFT_X, SPEAKER_BOTTOM_LEFT_Y, (44.5+17)*25.4])
speaker_xyz = (speaker_xyz_bottom_right+speaker_xyz_bottom_left+speaker_xyz_top_right+speaker_xyz_top_left)/4

In [ ]:
walls = None
x_min = - 4000
x_max = 500
y_min = -4000
y_max = 2000

In [ ]:
for idx, mic in enumerate(mic_xyzs):
    plt.scatter(mic[0], mic[1], label=f'mic {idx+1}')

plt.title("Répartition des micros")
plt.xlim([-5500, 1000])
plt.ylim([-4500, 2500])
plt.legend(loc="lower left", title="microphone")

In [ ]:
rirs_1 = np.load('../hackathon-minarm-2024/data/LivingRoom_preprocessed_hack/Human1/deconvoled_trim.npy')
rirs_2 = np.load('../hackathon-minarm-2024/data/LivingRoom_preprocessed_hack/Human2/deconvoled_trim.npy')
rirs = np.append(rirs_1, rirs_2, axis=0)

In [ ]:
localisations_1 = np.load('../hackathon-minarm-2024/data/LivingRoom_preprocessed_hack/Human1/centroid.npy')
localisations_2 = np.load('../hackathon-minarm-2024/data/LivingRoom_preprocessed_hack/Human2/centroid.npy')
localisations = np.append(localisations_1, localisations_2, axis=0)

In [ ]:
print(rirs.shape)
print(localisations.shape)
N_datapoints, N_mics, N_samples = rirs.shape
print(N_samples)

In [ ]:
rms = np.sqrt(np.square(rirs).mean(axis=2))

In [ ]:
print(rms)

In [ ]:
def distance(actual, expected):
    diff_x = actual[:, 0] - expected[:, 0]
    diff_y = actual[:, 1] - expected[:, 1]
    return np.sqrt(np.square(diff_x) + np.square(diff_y))

def mean_distance(actual, expected):
    return distance(actual, expected).mean()

In [ ]:
from sklearn.model_selection import train_test_split

X = pd.DataFrame(rms)
y = pd.DataFrame(localisations)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

In [ ]:
from sklearn.linear_model import LinearRegression

knn = LinearRegression()
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))